In [13]:
from src.generation.evolution import ModifiedNoveltyArchieve
from PIL import Image
from sklearn_extra.cluster import KMedoids
import cv2
import os
import numpy as np
import torch
import random
from data.swarmset import ContinuingDataset, SwarmDataset
from src.networks.embedding import NoveltyEmbedding
from src.generation.evolution import ModifiedHaltingEvolution
from src.networks.archive import DataAggregationArchive
from src.hil.HIL import HIL
import time
from src.networks.ensemble import Ensemble
from src.networks.network_wrapper import NetworkWrapper, ResNetWrapper
from src.constants import DEFAULT_OUTPUT_CONFIG, TWO_SENSOR_GENE_MODEL, SINGLE_SENSOR_GENE_MODEL, SINGLE_SENSOR_WORLD_CONFIG, TWO_SENSOR_WORLD_CONFIG, \
    SINGLE_SENSOR_HETEROGENEOUS_MODEL, SINGLE_SENSOR_HETEROGENEOUS_WORLD_CONFIG_AGNOSTIC, SINGLE_SENSOR_HETEROGENEOUS_WORLD_CONFIG_AWARE, HETEROGENEOUS_SUBGROUP_BEHAVIOR
from novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from novel_swarms.config.defaults import ConfigurationDefaults

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

def embed(image, behavior, network, concat_behavior=False, size=50, strategy="Mattson_and_Brown", species_aware=False):
    image = resizeInput(image, size)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    embedding = None
    if network is not None:
        if strategy == "Mattson_and_Brown":
            network.network.eval()
            if species_aware:
                embedding = network.colored_input(image)
            else:
                image = torch.from_numpy(image).to(device).float()
                embedding = network.network(image.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        if strategy == "ResNet":
            if species_aware:
                embedding = network.forward(image).detach().cpu().squeeze(dim=0).numpy()
            else:
                embedding = network.forward(image, greyscale=True).detach().cpu().squeeze(dim=0).numpy()
    else:
        embedding = behavior

    if concat_behavior:
        embedding = np.concatenate((embedding, behavior))

    return embedding


def execute_from_model(MODEL):
    out_name = MODEL['out_name']
    gecco_network = NetworkWrapper(output_size=5, lr=0.0, margin=0.0, weight_decay=1e-6, new_model=True,
                             manual_schedulers=True, dynamic_lr=True, warmup=5)
    gecco_network.load_from_path(MODEL["checkpoint"])
    gecco_network.eval_mode()

    res_network = ResNetWrapper()
    res_network.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


    out_config = DEFAULT_OUTPUT_CONFIG
    config = MODEL["config"]
    evolution = ModifiedHaltingEvolution(
        evolution_config=MODEL["g_e"],
        world=MODEL["g_e"].world_config,
        output_config=out_config,
        heterogeneous=MODEL["heterogeneous"],
    )
    evolution.restart_screen()

    print("Generating Genomes...")
    random.seed(config["seed"])
    np.random.seed(config["seed"])
    genome_set = []
    for i in range(config["generations"] * config["population"]):
        r_g = MODEL["g_e"].gene_builder.fetch_random_genome()
        genome_set.append(r_g)
    print(f"Genome 0: {genome_set[0]}")
    print(f"{len(genome_set)} Genomes Generated.")

    archives = [ModifiedNoveltyArchieve() for _ in range(8)]

    for i, genome in enumerate(genome_set):
        print(f"Generation: {i // config['population']} Population Member: {i % config['population']}")
        # The collection of the original behavior vector below is only used to collect data to compare with the baseline

        # AGNOSTIC SET
        # Set output to be agnostic
        evolution.output_configuration.colored = False
        evolution.behavior_discovery.world_config = SINGLE_SENSOR_HETEROGENEOUS_WORLD_CONFIG_AGNOSTIC
        evolution.behavior_discovery.world_config.behavior = ConfigurationDefaults.BEHAVIOR_VECTOR
        evolution.behavior_discovery.world_config.seed = config["seed"]
        print(genome)
        visual_behavior, _, baseline_behavior = evolution.simulate_specific(genome)

        # Brown_et_al, type-agnostic (REPR 1)
        embedding_A = baseline_behavior[-5:]
        archives[0].addToArchive(embedding_A, genome)

        # Mattson_and_Brown, type-agnostic (REPR 2)
        embedding_B = embed(visual_behavior, baseline_behavior, gecco_network,
                              concat_behavior=False,
                              strategy="Mattson_and_Brown", species_aware=False)
        archives[1].addToArchive(embedding_B, genome)

        # Combined 1 + 2, type-agnostic (REPR 3)
        embedding_C = embed(visual_behavior, baseline_behavior, gecco_network,
                              concat_behavior=True,
                              strategy="Mattson_and_Brown", species_aware=False)
        archives[2].addToArchive(embedding_C, genome)

        # ResNet, type-agnostic (REPR 4)
        embedding_D = embed(visual_behavior, baseline_behavior, res_network,
                              concat_behavior=False,
                              strategy="ResNet", species_aware=False)
        archives[3].addToArchive(embedding_D, genome)

        # AWARE SET
        # Set output to be aware
        evolution.output_configuration.colored = True
        evolution.behavior_discovery.world_config = SINGLE_SENSOR_HETEROGENEOUS_WORLD_CONFIG_AWARE
        evolution.behavior_discovery.world_config.behavior = HETEROGENEOUS_SUBGROUP_BEHAVIOR
        evolution.behavior_discovery.world_config.seed = config["seed"]
        visual_behavior, _, baseline_behavior = evolution.simulate_specific(genome)

        # Brown_et_al, type-aware (REPR 5)
        embedding_E = baseline_behavior
        archives[4].addToArchive(embedding_E, genome)

        # Mattson_and_Brown, type-aware (REPR 6)
        embedding_F = embed(visual_behavior, baseline_behavior, gecco_network,
                              concat_behavior=False,
                              strategy="Mattson_and_Brown", species_aware=True)
        archives[5].addToArchive(embedding_F, genome)

        # Combined 1 + 2, type-aware (REPR 7)
        embedding_G = embed(visual_behavior, baseline_behavior, gecco_network,
                              concat_behavior=True,
                              strategy="Mattson_and_Brown", species_aware=True)
        archives[6].addToArchive(embedding_G, genome)

        # ResNet, type-aware (REPR 8)
        embedding_H = embed(visual_behavior, baseline_behavior, res_network,
                              concat_behavior=False,
                              strategy="ResNet", species_aware=True)
        archives[7].addToArchive(embedding_H, genome)

    for i, archive_i in enumerate(archives):
        archive_i.saveGenotypes(f"{out_name}_repr_{i + 1}__g")
        archive_i.saveArchive(f"{out_name}_repr_{i + 1}__b")
    print("Completed Model.")

In [16]:
gene_builder = SINGLE_SENSOR_HETEROGENEOUS_MODEL

SEED = 2
AGENTS = 24
GEN, POP = 50, 100
CR, MR = 0.7, 0.15
K = 12
lifespan = 1200

model = {
    "out_name": 'R-s1',
    "checkpoint": '../../../checkpoints/gecco.pt',
    "heterogeneous": True,
    "requires_image_out": True,
    "config": {
        "generations": GEN,
        "population": POP,
        "lifespan": lifespan,
        "agents": AGENTS,
        "seed": SEED,
        "k": K,
        "crossover_rate": CR,
        "mutation_rate": MR,
    },
    "g_e": GeneticEvolutionConfig(
        gene_builder=gene_builder,
        phenotype_config=HETEROGENEOUS_SUBGROUP_BEHAVIOR,
        world_config=SINGLE_SENSOR_HETEROGENEOUS_WORLD_CONFIG_AGNOSTIC,
        n_generations=GEN,
        n_population=POP,
        crossover_rate=CR,
        mutation_rate=MR,
        k_nn=K,
        simulation_lifespan=lifespan,
        display_novelty=False,
        save_archive=False,
        show_gui=True,
        use_external_archive=False,
        seed=SEED,
    )
}

execute_from_model(model)

ResNet Init with 512
Generating Genomes...
Genome 0: [-0.9, -0.8, -0.8, 0.1, -0.5, -0.1, -0.2, 0.9, 0.3333333333333333]
5000 Genomes Generated.
Generation: 0 Population Member: 0
[-0.9, -0.8, -0.8, 0.1, -0.5, -0.1, -0.2, 0.9, 0.3333333333333333]
[-0.9, -0.8, -0.8, 0.1, -0.5, -0.1, -0.2, 0.9, 0.3333333333333333]
[<novel_swarms.config.AgentConfig.DiffDriveAgentConfig object at 0x7fbe781e0c40>, <novel_swarms.config.AgentConfig.DiffDriveAgentConfig object at 0x7fbe7823f250>]
torch.Size([200, 200])
Final Shape: torch.Size([1, 3, 200, 200])
[-0.9, -0.8, -0.8, 0.1, -0.5, -0.1, -0.2, 0.9, 0.3333333333333333]
[<novel_swarms.config.AgentConfig.DiffDriveAgentConfig object at 0x7fbe65bcc940>, <novel_swarms.config.AgentConfig.DiffDriveAgentConfig object at 0x7fbe78360bb0>]
torch.Size([1, 3, 200, 200])
Final Shape: torch.Size([1, 3, 200, 200])
Generation: 0 Population Member: 1
[0.9, -0.9, 0.8, -0.5, 0.3, 1.0, 0.2, 0.6, 0.25]
[0.9, -0.9, 0.8, -0.5, 0.3, 1.0, 0.2, 0.6, 0.25]
[<novel_swarms.config.Age